<a href="https://colab.research.google.com/github/06Priya-s/PRODIGY_GA_04/blob/main/Training_model_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Install necessary libraries
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import gradio as gr
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling
from transformers import create_optimizer
from datasets import load_dataset
import tensorflow as tf
import os

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [15]:
# Load custom dataset
file_path = "/content/drive/MyDrive/ai_dataset.txt"

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()
# Simple split into lines or paragraphs
examples = [line.strip() for line in text.split("\n") if line.strip()]
inputs = tokenizer(examples, return_tensors="tf", padding="max_length", truncation=True, max_length=128)
# Load dataset from plain text file
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )

# Pass the correct file_path to your function
train_dataset = load_dataset(file_path, tokenizer)



/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [16]:
# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [17]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-ai-finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
)


In [18]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

In [19]:
# Fine-tune the model
trainer.train()
trainer.save_model()
tokenizer.save_pretrained("./gpt2-ai-finetuned")

print("Fine-tuning complete!")


Step,Training Loss


Fine-tuning complete!


In [20]:
# Text Generation
# Load the fine-tuned model for inference
model = GPT2LMHeadModel.from_pretrained("./gpt2-ai-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-ai-finetuned")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [21]:
# Text generation function
def generate_text(prompt, max_length=100, temperature=0.7):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [22]:
# Launch Gradio interface
gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Enter Prompt", placeholder="e.g. Artificial Intelligence is"),
        gr.Slider(50, 300, step=10, label="Max Length", value=100),
        gr.Slider(0.1, 1.5, step=0.1, label="Temperature", value=0.7),
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="Fine-Tuned GPT-2 Text Generator",
    description="Generate text using your custom fine-tuned GPT-2 model on AI-related data.",
    theme="default"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a640a58b427138df47.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
